In [ ]:
#pip install psycopg2-binary
!pip install sqlalchemy


In [ ]:
import psycopg2
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:


# params PostgreSQL
db_params = {
    'dbname': 'DBInmerso',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

# table

tables_to_save = ['usuarios', 'discapacidad', 'renta', 'patrimonio', 'familia_numerosa', 'disponibilidad', 'historial_usuarios', 'tipo_resolucion', 'solicitudes', 'hoteles', 'destinos', 'resolucion_solicitudes', 'scoring']


# Connection
connection = psycopg2.connect(**db_params)

# Cursor
cursor = connection.cursor()


for table in tables_to_save:
    # DataFrame
    sql_query = f"SELECT * FROM {table};"
    df = pd.read_sql_query(sql_query, connection)

    # Save DataFrame en CSV-file
    filename = f'{table}.csv'
    df.to_csv(filename, index=False)
    print(f'Data from table {table} saved to {filename}')

# Close
cursor.close()
connection.close()




In [ ]:
usuarios = pd.read_csv('usuarios.csv')
discapacidad = pd.read_csv('discapacidad.csv')
renta = pd.read_csv('renta.csv')
patrimonio = pd.read_csv('patrimonio.csv')
familia_numerosa = pd.read_csv('familia_numerosa.csv')

In [ ]:
# SCORE EDAD


# add score

for index, row in usuarios.iterrows():
    edad = row['edad']
    SCORE = 0
    if edad < 60:
        SCORE = 1
    elif edad == 60:
        SCORE = 2
    elif edad <= 78:
        SCORE = 2 + (edad - 60)
    else:
        SCORE = 20
    
    usuarios.at[index, 'score'] = SCORE

usuarios['score'] = usuarios['score'].astype(int)


print(usuarios)

In [ ]:
#SCORE DISCAPACIDAD


# add score
for index, row in discapacidad.iterrows():
    grado_dis = row['grado_dis']
    SCORE = 0
    if grado_dis == 1:
        SCORE = 10
    else:
        grado_dis == 2
        SCORE = 15

    
    discapacidad.at[index, 'score'] = SCORE

discapacidad['score'] = discapacidad['score'].astype(int)

print(discapacidad)

In [ ]:
# SCORE renta
#merge USUARIOS, RENTA, SOLICITUDES

result_temp = pd.merge(usuarios, renta, on='dni', how='left')
renta_score = pd.merge(result_temp, solicitudes, on='id_solicitud')

In [ ]:
# renta



# add score

for usuarios_sol in renta_score['usuarios_sol']:
    if usuarios_sol > 1:
        for index, row in renta_score.iterrows():
            renta_sol = row['renta_sol']
            SCORE = 0
            if renta_sol <= 484.61:
                SCORE = 50
            elif 484.61 < renta_sol <= 900:
                SCORE = 45
            elif 900 < renta_sol <= 1050:
                SCORE = 40
            elif 1050 < renta_sol <= 1200:
                SCORE = 35
            elif 1200 < renta_sol <= 1350:
                SCORE = 30
            elif 1350 < renta_sol <= 1500:
                SCORE = 25
            elif 1500 < renta_sol <= 1650:
                SCORE = 20
            elif 1650 < renta_sol <= 1800:
                SCORE = 15
            elif 1800 < renta_sol <= 1950:
                SCORE = 10
            elif 1950 < renta_sol <= 2100:
                SCORE = 5
            else:
                SCORE = 0
    else:
        for index, row in renta_score.iterrows():
            renta = row['renta']
            SCORE = 0
            if renta <= 484.61:
                SCORE = 50
            elif 484.61 < renta <= 900:
                SCORE = 45
            elif 900 < renta <= 1050:
                SCORE = 40
            elif 1050 < renta <= 1200:
                SCORE = 35
            elif 1200 < renta <= 1350:
                SCORE = 30
            elif 1350 < renta <= 1500:
                SCORE = 25
            elif 1500 < renta <= 1650:
                SCORE = 20
            elif 1650 < renta <= 1800:
                SCORE = 15
            elif 1800 < renta <= 1950:
                SCORE = 10
            elif 1950 < renta <= 2100:
                SCORE = 5
            else:
                SCORE = 0
        


    renta_score.at[index, 'renta_score'] = SCORE

renta_score['renta_score'] = renta_score['renta_score'].astype(int)

print(renta_score)




In [ ]:
# SCORE patrimonio
#merge USUARIOS, patrimonio, SOLICITUDES

result_temp_pat = pd.merge(usuarios, patrimonio, on='dni', how='left')
patrimonio_score = pd.merge(result_temp_pat, solicitudes, on='id_solicitud')

In [ ]:
# patrimonio



# add score

for usuarios_sol in patrimonio_score['usuarios_sol']:
    if usuarios_sol > 1:
        for index, row in patrimonio_score.iterrows():
            patrimonio_sol = row['patrimonio_sol']
            SCORE = 0
            if patrimonio_sol <= 20000:
                SCORE = 150
            elif 20000 < patrimonio_sol <= 40000:
                SCORE = 100
            elif 40000 < patrimonio_sol <= 80000:
                SCORE = 55
            elif 80000 < patrimonio_sol <= 130000:
                SCORE = 45
            elif 130000 < patrimonio_sol <= 170000:
                SCORE = 35
            elif 170000 < patrimonio_sol <= 230000:
                SCORE = 25
            elif 230000 < patrimonio_sol <= 275000:
                SCORE = 15
            elif 275000 < patrimonio_sol <= 325000:
                SCORE = 5
            else:
                SCORE = 0
    else:
        for index, row in patrimonio_score.iterrows():
            patrimonio = row['patrimonio']
            SCORE = 0
            if patrimonio <= 20000:
                SCORE = 150
            elif 20000 < patrimonio <= 40000:
                SCORE = 100
            elif 40000 < patrimonio <= 80000:
                SCORE = 55
            elif 80000 < patrimonio <= 130000:
                SCORE = 45
            elif 130000 < patrimonio <= 170000:
                SCORE = 35
            elif 170000 < patrimonio <= 230000:
                SCORE = 25
            elif 230000 < patrimonio <= 275000:
                SCORE = 15
            elif 275000 < patrimonio <= 325000:
                SCORE = 5
            else:
                SCORE = 0
        


    patrimonio_score.at[index, 'patrimonio_score'] = SCORE

patrimonio_score['patrimonio_score'] = patrimonio_score['patrimonio_score'].astype(int)

print(patrimonio_score)

In [ ]:
# familia_numerosa



# add score
for index, row in familia_numerosa.iterrows():
    tipo_fam_num = row['tipo_fam_num']
    SCORE = 0
    if tipo_fam_num == 1:
        SCORE = 5
    elif tipo_fam_num == 2:
        SCORE = 10

    
    familia_numerosa.at[index, 'score'] = SCORE

familia_numerosa['score'] = familia_numerosa['score'].astype(int)

print(familia_numerosa)


#merge
fam_num__score = pd.merge(familia_numerosa, usuarios, on='dni')
